<a href="https://colab.research.google.com/github/rajlm10/Shoppster/blob/main/MLD_Project_Triplet_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Data from Kaggle

In [ ]:
!mkdir ~/.kaggle/
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

 96% 545M/565M [00:08<00:00, 81.6MB/s]
100% 565M/565M [00:08<00:00, 68.3MB/s]


In [ ]:
!unzip -qq /content/fashion-product-images-small.zip 

In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
data = pd.read_csv('/content/styles.csv',engine='python',on_bad_lines='skip')

In [ ]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44424 non-null  int64  
 1   gender              44424 non-null  object 
 2   masterCategory      44424 non-null  object 
 3   subCategory         44424 non-null  object 
 4   articleType         44424 non-null  object 
 5   baseColour          44409 non-null  object 
 6   season              44403 non-null  object 
 7   year                44423 non-null  float64
 8   usage               44107 non-null  object 
 9   productDisplayName  44417 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.4+ MB


In [ ]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44077 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44077 non-null  int64  
 1   gender              44077 non-null  object 
 2   masterCategory      44077 non-null  object 
 3   subCategory         44077 non-null  object 
 4   articleType         44077 non-null  object 
 5   baseColour          44077 non-null  object 
 6   season              44077 non-null  object 
 7   year                44077 non-null  float64
 8   usage               44077 non-null  object 
 9   productDisplayName  44077 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.7+ MB


#Clean, Filter and Analyze Data

In [ ]:
articles_to_keep= [article for article,_ in  Counter(data['articleType']).most_common(22)]

In [ ]:
data = data[data['articleType'].isin(articles_to_keep)].reset_index(drop=True)
data = data[data['gender'].isin(["Men","Women"])].reset_index(drop=True)
data = data[data['masterCategory']!='Free Items'].reset_index(drop=True)
data = data[data['usage'].isin(["Casual","Sports","Formal","Ethnic"])].reset_index(drop=True)


In [ ]:
Counter(data['gender'])

Counter({'Men': 19494, 'Women': 11680})

In [ ]:
Counter(data['masterCategory'])

Counter({'Apparel': 15224,
         'Accessories': 7367,
         'Footwear': 7990,
         'Personal Care': 593})

In [ ]:
Counter(data['subCategory']).most_common(25)

[('Topwear', 12846),
 ('Shoes', 6499),
 ('Watches', 2351),
 ('Bags', 1779),
 ('Bottomwear', 1529),
 ('Eyewear', 920),
 ('Wallets', 901),
 ('Innerwear', 847),
 ('Belts', 801),
 ('Sandal', 748),
 ('Flip Flops', 743),
 ('Socks', 614),
 ('Fragrance', 588),
 ('Perfumes', 6),
 ('Loungewear and Nightwear', 2)]

In [ ]:
data.loc[data['subCategory']=='Loungewear and Nightwear','subCategory'] = 'Innerwear'

In [ ]:
data.loc[data['subCategory']=='Perfumes','subCategory'] = 'Fragrance'

In [ ]:
Counter(data['subCategory']).most_common(25)

[('Topwear', 12846),
 ('Shoes', 6499),
 ('Watches', 2351),
 ('Bags', 1779),
 ('Bottomwear', 1529),
 ('Eyewear', 920),
 ('Wallets', 901),
 ('Innerwear', 849),
 ('Belts', 801),
 ('Sandal', 748),
 ('Flip Flops', 743),
 ('Socks', 614),
 ('Fragrance', 594)]

In [ ]:
Counter(data['usage']).most_common(25)

[('Casual', 24025), ('Sports', 3251), ('Formal', 1990), ('Ethnic', 1908)]

In [ ]:
articles_to_keep

['Tshirts',
 'Shirts',
 'Casual Shoes',
 'Watches',
 'Sports Shoes',
 'Kurtas',
 'Tops',
 'Handbags',
 'Heels',
 'Sunglasses',
 'Wallets',
 'Flip Flops',
 'Sandals',
 'Briefs',
 'Belts',
 'Backpacks',
 'Socks',
 'Formal Shoes',
 'Jeans',
 'Perfume and Body Mist',
 'Shorts',
 'Trousers']

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
descriptions_without_gender = []
for idx,item in tqdm(data.iterrows(),total=data.shape[0]):
  try:
    data.loc[idx,'productDisplayName'] = item['productDisplayName'][item['productDisplayName'].index(item['gender']):]
  except:
    descriptions_without_gender.append(idx)


  0%|          | 0/31174 [00:00<?, ?it/s]

In [ ]:
len(descriptions_without_gender)

874

In [ ]:
data = data.drop(labels=descriptions_without_gender,axis=0).reset_index(drop=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30300 entries, 0 to 30299
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  30300 non-null  int64  
 1   gender              30300 non-null  object 
 2   masterCategory      30300 non-null  object 
 3   subCategory         30300 non-null  object 
 4   articleType         30300 non-null  object 
 5   baseColour          30300 non-null  object 
 6   season              30300 non-null  object 
 7   year                30300 non-null  float64
 8   usage               30300 non-null  object 
 9   productDisplayName  30300 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.3+ MB


In [ ]:
first_word = []

for idx,item in tqdm(data.iterrows(),total=data.shape[0]):
  try:
    first_word.append(item['productDisplayName'].split(' ')[0])
  except:
    pass

Counter(first_word)

  0%|          | 0/30300 [00:00<?, ?it/s]

Counter({'Men': 16675,
         'Women': 10717,
         'Mens': 185,
         "Men's": 2236,
         "Women's": 444,
         'Womens': 37,
         'MenCasual': 1,
         'Women.': 1,
         'MenNavy': 1,
         'MenLime': 1,
         'MenSpeeder': 1,
         'MenShape-Ups': 1})

In [ ]:
for idx,item in tqdm(data.iterrows(),total=data.shape[0]):

  try:
    if 'Mens' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("Mens","Men")

    if "Men's" in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("Men's","Men")

    if 'MenCasual' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("MenCasual","Men")

    if 'MenNavy' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("MenNavy","Men")

    if 'MenLime' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("MenLime","Men")

    if 'MenSpeeder' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("MenSpeeder","Men")

    if 'MenShape-Ups' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("MenShape-Ups","Men")

    if "Women's" in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("Women's","Women")
    
    if 'Womens' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("Womens","Women")

    if 'Women.' in item['productDisplayName']:
      data.loc[idx,'productDisplayName'] = item['productDisplayName'].replace("Women.","Women")
    
  except:
    pass

first_word = []

for idx,item in tqdm(data.iterrows(),total=data.shape[0]):
  try:
    first_word.append(item['productDisplayName'].split(' ')[0])
  except:
    pass

Counter(first_word)

  0%|          | 0/30300 [00:00<?, ?it/s]

  0%|          | 0/30300 [00:00<?, ?it/s]

Counter({'Men': 19101, 'Women': 11199})

In [ ]:
a='Men Navy Blue Shirt'.split(' ')
a.insert(1,'Casual')
a=" ".join(a)
a

'Men Casual Navy Blue Shirt'

In [ ]:
descriptions_without_gender = []
for idx,item in tqdm(data.iterrows(),total=data.shape[0]):
  try:
    s = item['productDisplayName'].split(' ')
    s.insert(1,item['usage'])
    s = " ".join(s)
    data.loc[idx,'productDisplayName'] = s
  except:
    descriptions_without_gender.append(idx)

descriptions_without_gender

  0%|          | 0/30300 [00:00<?, ?it/s]

[]

In [ ]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Men Casual Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Men Casual Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Women Casual Silver Watch
3,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Men Casual Grey T-shirt
4,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Men Casual Chain Reaction T-shirt


In [ ]:
for value in set(data['articleType']):
  print(value)
  print(data[data['articleType']==value].groupby(by='gender')['gender'].count())
  print()

Sports Shoes
gender
Men      1542
Women     385
Name: gender, dtype: int64

Briefs
gender
Men      557
Women    242
Name: gender, dtype: int64

Tops
gender
Men         1
Women    1370
Name: gender, dtype: int64

Watches
gender
Men      1449
Women     874
Name: gender, dtype: int64

Shorts
gender
Men      335
Women     58
Name: gender, dtype: int64

Formal Shoes
gender
Men    627
Name: gender, dtype: int64

Backpacks
gender
Men      53
Women    32
Name: gender, dtype: int64

Handbags
gender
Men         4
Women    1628
Name: gender, dtype: int64

Flip Flops
gender
Men      493
Women    247
Name: gender, dtype: int64

Jeans
gender
Men      314
Women    221
Name: gender, dtype: int64

Perfume and Body Mist
gender
Men      323
Women    268
Name: gender, dtype: int64

Casual Shoes
gender
Men      2231
Women     327
Name: gender, dtype: int64

Sandals
gender
Men      745
Women      2
Name: gender, dtype: int64

Belts
gender
Men      538
Women    263
Name: gender, dtype: int64

Shirts
gender
M

In [ ]:
for value in set(data['articleType']):
  print(value)
  print(data[data['articleType']==value].groupby(by='gender')['gender'].count())
  print()

Sports Shoes
gender
Men      1542
Women     385
Name: gender, dtype: int64

Briefs
gender
Men      557
Women    242
Name: gender, dtype: int64

Tops
gender
Men         1
Women    1370
Name: gender, dtype: int64

Watches
gender
Men      1449
Women     874
Name: gender, dtype: int64

Shorts
gender
Men      335
Women     58
Name: gender, dtype: int64

Formal Shoes
gender
Men    627
Name: gender, dtype: int64

Backpacks
gender
Men      53
Women    32
Name: gender, dtype: int64

Handbags
gender
Men         4
Women    1628
Name: gender, dtype: int64

Flip Flops
gender
Men      493
Women    247
Name: gender, dtype: int64

Jeans
gender
Men      314
Women    221
Name: gender, dtype: int64

Perfume and Body Mist
gender
Men      323
Women    268
Name: gender, dtype: int64

Casual Shoes
gender
Men      2231
Women     327
Name: gender, dtype: int64

Sandals
gender
Men      745
Women      2
Name: gender, dtype: int64

Belts
gender
Men      538
Women    263
Name: gender, dtype: int64

Shirts
gender
M

In [ ]:
colour_not_in_product_descp = []

for idx,item in tqdm(data.iterrows(),total = data.shape[0]):
  if item['gender'] in item['productDisplayName']:
    pass
  
  else:
    colour_not_in_product_descp.append(idx)

colour_not_in_product_descp



  0%|          | 0/30300 [00:00<?, ?it/s]

[]

In [ ]:
for value in set(data['articleType']):
  print(value)
  print(data[data['articleType']==value].groupby(by='baseColour')['baseColour'].count())
  print()

Sports Shoes
baseColour
Beige                  3
Black                521
Blue                  89
Bronze                 1
Brown                 23
Charcoal               7
Cream                  1
Fluorescent Green      2
Green                  9
Grey                 209
Khaki                  1
Lavender               1
Maroon                 2
Multi                  1
Navy Blue             37
Olive                  4
Orange                 9
Pink                  12
Purple                16
Red                   30
Sea Green              1
Silver                72
White                868
Yellow                 8
Name: baseColour, dtype: int64

Briefs
baseColour
Beige               2
Black             131
Blue               68
Brown              22
Burgundy            5
Charcoal           39
Cream               5
Gold                2
Green              17
Grey               88
Grey Melange       48
Khaki               4
Lavender            5
Maroon             15
Mauve             

#Triplet Mining

In [ ]:
triplet_map={"Trousers":{"semi":{"articles":["Trousers","Shorts","Jeans"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Trousers"],"gender":"same","baseColour":"same"}
                         },
             
             "Kurtas":{"semi":{"articles":["Kurtas","Tshirts","Shirts"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Kurtas"],"gender":"same","baseColour":"same"}
                         },
             
             "Shorts":{"semi":{"articles":["Shorts","Trousers","Jeans"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Shorts"],"gender":"same","baseColour":"same"}
                         },
             
             "Casual Shoes":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Casual Shoes"],"gender":"same","baseColour":"same"}
                         },
             
             "Formal Shoes":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Formal Shoes"],"gender":"same","baseColour":"same"}
                         },
             
             "Sports Shoes":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Sports Shoes"],"gender":"same","baseColour":"same"}
                         },
             
             "Flip Flops":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Flip Flops","Sandals"],"gender":"same","baseColour":"same"}
                         },
             
             "Sandals":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Flip Flops","Sandals","Heels"],"gender":"same","baseColour":"same"}
                         },
             
             "Heels":{"semi":{"articles":["Casual Shoes","Formal Shoes","Sports Shoes","Flip Flops"],"gender":"same","baseColour":"different"},
                         "hard":{"articles":["Heels","Sandals"],"gender":"same","baseColour":"same"}
                         },
             
             "Briefs":{"semi":{"articles":["Briefs","Shorts"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Briefs"],"gender":"same","baseColour":"same"}
                         },
             
             "Backpacks":{"semi":{"articles":["Wallets","Handbags"],"gender":"same","baseColour":"different"},
                         "hard":{"articles":["Backpacks"],"gender":"same","baseColour":"same"}
                         },
             
             "Belts":{"semi":{"articles":["Belts","Wallets"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Belts"],"gender":"same","baseColour":"same"}
                         },
             
             "Handbags":{"semi":{"articles":["Backpacks","Wallets","Handbags"],"gender":"same","baseColour":"different"},
                         "hard":{"articles":["Handbags"],"gender":"same","baseColour":"same"}
                         },
             
             "Socks":{"semi":{"articles":["Socks","Flip Flops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Socks","Sandals"],"gender":"same","baseColour":"same"}
                         },
             
             "Watches":{"semi":{"articles":["Watches","Wallets"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Watches"],"gender":"same","baseColour":"same"}
                         },
             
             "Tshirts":{"semi":{"articles":["Shirts","Tshirts","Tops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Tshirts"],"gender":"same","baseColour":"same"}
                         },
             
             "Shirts":{"semi":{"articles":["Shirts","Tshirts","Tops"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Shirts"],"gender":"same","baseColour":"same"}
                         },
             
             "Tops":{"semi":{"articles":["Shirts","Tshirts"],"gender":"same","baseColour":"different"},
                         "hard":{"articles":["Tops"],"gender":"same","baseColour":"same"}
                         },
             
             "Perfume and Body Mist":{"semi":{"articles":["Perfume and Body Mist"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Perfume and Body Mist"],"gender":"same","baseColour":"same"}
                         },
             
             "Wallets":{"semi":{"articles":["Belts","Wallets","Watches"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Wallets"],"gender":"same","baseColour":"same"}
                         },
             
             "Jeans":{"semi":{"articles":["Jeans","Trousers","Shorts"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Jeans"],"gender":"same","baseColour":"same"}
                         },
             
             "Sunglasses":{"semi":{"articles":["Belts","Wallets","Sunglasses"],"gender":"opposite","baseColour":"different"},
                         "hard":{"articles":["Sunglasses"],"gender":"same","baseColour":"same"}
                         },
                          

             }

In [ ]:
data.iloc[:2,:]

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Men Casual Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Men Casual Party Blue Jeans


In [ ]:
male_data = data[data['gender']=='Men']
female_data = data[data['gender']=='Women']
male_data.shape[0],female_data.shape[0]

(19101, 11199)

In [ ]:
set(data['masterCategory'])-{'Accessories'}

{'Apparel', 'Footwear', 'Personal Care'}

In [ ]:
import random
 
random_seed = 1


def gen_triplets(data,male_data,female_data,
                 num_semi_negatives = 4,
                 num_hard_negatives = 3):
  
  triples= []
  colours = set(data['baseColour'])

  exceptions = []

  for idx, item in tqdm(data.iterrows(), total=data.shape[0]):
    try:
      masterCategory = item['masterCategory']
      articleType = item['articleType']
      gender = item['gender']
      baseColour = item['baseColour']
      easy_samples = list(set(data['masterCategory'])-{masterCategory})
      
      for category in easy_samples:
        triple={}
        easy_negative=data[data['masterCategory']==category].sample(1,random_state=random_seed)['productDisplayName'].tolist()[0]
        
      
        triple["anchor"]=item['id']
        triple["positive"]=item['productDisplayName']
        triple["negative"] = easy_negative
        triple["level"] = "easy"
        triples.append(triple)
      
      gender_sample = triplet_map[articleType]["semi"]["gender"]
      gender_possibility = gender if gender_sample=="same" else list({"Men","Women"}-{gender})[0]
      colour_sample = triplet_map[articleType]["semi"]["baseColour"]
      colour_possibilities = [baseColour] if colour_sample=="same" else list(colours - {baseColour})

      article_possibilities = triplet_map[articleType]["semi"]["articles"]

      
      semi_negatives = data[(data['gender']==gender_possibility) & (data['baseColour'].isin(colour_possibilities))
                            & (data['articleType'].isin(article_possibilities))]

      loop_count=0

      while(True):
        loop_count+=1
        semi_negatives = semi_negatives.sample(n=num_semi_negatives,random_state=random_seed)
        if item['id'] not in set(semi_negatives['id']) or loop_count==10:
          break

      semi_negatives = semi_negatives['productDisplayName'].tolist()

      
      for semi_negative in semi_negatives:
        triple={}       
        triple["anchor"]=item['id']
        triple["positive"]=item['productDisplayName']
        triple["negative"] = semi_negative
        triple["level"] = "semi"
        triples.append(triple)


      # ##Hard Negatives

      gender_sample = triplet_map[articleType]["hard"]["gender"]
      gender_possibility = gender if gender_sample=="same" else list({"Men","Women"}-{gender})[0]
      colour_sample = triplet_map[articleType]["hard"]["baseColour"]
      colour_possibilities = [baseColour] if colour_sample=="same" else list(colours - {baseColour})

      article_possibilities = triplet_map[articleType]["hard"]["articles"]


      
      hard_negatives = data[(data['gender']==gender_possibility) & (data['baseColour'].isin(colour_possibilities))
                            & (data['articleType'].isin(article_possibilities))]

      loop_count=0

      while(True):
        loop_count+=1
        hard_negatives = hard_negatives.sample(n=num_hard_negatives,random_state=random_seed)
        if item['id'] not in set(hard_negatives['id']) or loop_count==10:
          break

      hard_negatives = hard_negatives['productDisplayName'].tolist()

      
      for hard_negative in hard_negatives:
        triple={}       
        triple["anchor"]=item['id']
        triple["positive"]=item['productDisplayName']
        triple["negative"] = hard_negative
        triple["level"] = "hard"
        triples.append(triple)

    except:
      exceptions.append(idx)
    
    


  return (pd.DataFrame(triples)),exceptions
   




In [ ]:
triplet_data,exceptions=gen_triplets(data,male_data,female_data)

  0%|          | 0/30300 [00:00<?, ?it/s]

In [ ]:
len(exceptions)

306

In [ ]:
triplet_data

,anchor,positive,negative,level
0,15970,Men Casual Navy Blue Shirt,Women Casual Clematis Perfume,easy
1,15970,Men Casual Navy Blue Shirt,Women Casual Valley Hoor Green Wallet,easy
2,15970,Men Casual Navy Blue Shirt,Men Casual Yellow Flip Flops,easy
3,15970,Men Casual Navy Blue Shirt,Women Casual Yellow Printed Top,semi
4,15970,Men Casual Navy Blue Shirt,Women Casual Light Winter Purple Tops,semi
...,...,...,...,...
302077,51623,Women Casual Pink Dial Chronograph Watch ES3050,Men Casual Silver Dial Watch,semi
302078,51623,Women Casual Pink Dial Chronograph Watch ES3050,Men Casual White Dial Watch,semi
302079,51623,Women Casual Pink Dial Chronograph Watch ES3050,Women Casual Pink Dial Watch,hard
302080,51623,Women Casual Pink Dial Chronograph Watch ES3050,Women Casual Pink Dial Chronograph Watch SX005...,hard


In [ ]:
triplet_data.drop_duplicates(inplace=True)

In [ ]:
triplet_data = triplet_data.reset_index(drop=True)

In [ ]:
least_common = Counter(triplet_data['anchor']).most_common()[-1]
least_common

(42936, 5)

#Final Data

In [ ]:
triplet_data

,anchor,positive,negative,level
0,15970,Men Casual Navy Blue Shirt,Women Casual Clematis Perfume,easy
1,15970,Men Casual Navy Blue Shirt,Women Casual Valley Hoor Green Wallet,easy
2,15970,Men Casual Navy Blue Shirt,Men Casual Yellow Flip Flops,easy
3,15970,Men Casual Navy Blue Shirt,Women Casual Yellow Printed Top,semi
4,15970,Men Casual Navy Blue Shirt,Women Casual Light Winter Purple Tops,semi
...,...,...,...,...
294967,51623,Women Casual Pink Dial Chronograph Watch ES3050,Men Casual Sliver Dial Chronograph Watch,semi
294968,51623,Women Casual Pink Dial Chronograph Watch ES3050,Men Casual Silver Dial Watch,semi
294969,51623,Women Casual Pink Dial Chronograph Watch ES3050,Men Casual White Dial Watch,semi
294970,51623,Women Casual Pink Dial Chronograph Watch ES3050,Women Casual Pink Dial Watch,hard


In [ ]:
triplet_data.to_csv("triplet_data.csv",index=False)

In [ ]:
pd.read_csv('/content/triplet_data.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294972 entries, 0 to 294971
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   anchor    294972 non-null  int64 
 1   positive  294972 non-null  object
 2   negative  294972 non-null  object
 3   level     294972 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.0+ MB
